In [1]:
# !pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
from model.hifigan.env import AttrDict
import easydict
import torch
import yaml
import json

device = torch.device('cpu')

args = easydict.EasyDict()
args['preprocess_config'] = './config/kss/preprocess.yaml'
args['synthesizer_config'] = './config/kss/model.yaml'
args['generator_config'] = './model/hifigan/config_v1.json'
args['train_config'] = './config/kss/train.yaml'

preprocess_config = yaml.load(open(args.preprocess_config, "r"), Loader=yaml.FullLoader)
synthesizer_config = yaml.load(open(args.synthesizer_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
with open(args.generator_config, "r", encoding='utf8') as j:
    generator_config = json.loads(j.read())
    generator_config = AttrDict(generator_config)

configs = (preprocess_config, [synthesizer_config, generator_config], train_config)



In [2]:
from model.jets import JETSSynthesizer

model = JETSSynthesizer(*configs)

In [5]:
cnt = 0 
nan = 0
for p in model.parameters():
    # print(p)
    cnt += 1

In [6]:
cnt

465

In [16]:
a = torch.BoolTensor([True,True,False])

a.sum()

tensor(2)